# Juntando Todos os Arquivos Pré-processados

#### Carlos Eduardo Cassimiro da Silva

Pré-processamos separadamente os dados das notas do Ideb, contagem de CVLI, estimativa da população do Ceará para 2019, o grafo montado e o GeoJSON do Ceará. Agora, precisamos juntar todos os dados e realizar os devidos ajustes faltantes.

**Módulos utilizados**

In [1]:
import pandas as pd
import json
import math

# 1. Abrindo os arquivos

In [92]:
nomes_municipios = pd.read_csv('arquivos/ideb_municipios.csv')
cvli = pd.read_csv('arquivos/cvli_fortaleza_2019_att.csv', encoding='ISO-8859-1')
populacao = pd.read_csv('arquivos/populacao_ceara_2019.csv')

In [3]:
with open('grafo_municipios_final.json', encoding='utf-8') as meu_json:
    grafo_municipios = json.load(meu_json)

In [4]:
with open('arquivos/ceara_prepross.geojson', encoding='utf-8') as meu_json:
    geojson_municipios = json.load(meu_json)

# 2. Visualizando o formato dos conjuntos de dados

In [5]:
geojson_municipios

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'id': '2300101',
    'name': 'ABAIARA',
    'description': 'Abaiara'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-38.962617023, -7.400213519],
      [-38.9699430065, -7.425970715],
      [-38.9802096335, -7.4256557701],
      [-39.0057514064, -7.4248702064],
      [-39.0085042986, -7.4247851007],
      [-39.0386107935, -7.4238568803],
      [-39.0442544358, -7.4236828229],
      [-39.0777823302, -7.4226461221],
      [-39.0840265059, -7.4066304771],
      [-39.0902377832, -7.383012034],
      [-39.0956411399, -7.3698438945],
      [-39.0949647958, -7.3635665806],
      [-39.1008719796, -7.3429301091],
      [-39.1010908316, -7.3414524542],
      [-39.1015549879, -7.3277226515],
      [-39.095480973, -7.3181456369],
      [-39.0847055514, -7.309352611],
      [-39.0812032038, -7.2984556808],
      [-39.0905850231, -7.281708887],
      [-39.0835354706, -7.2786909717],
      [-39.0817136843, -

In [6]:
cvli

,MUNICIPIO,QTD_CVLI
0,ABAIARA,0
1,ACARAPE,6
2,ACARAU,8
3,ACOPIARA,14
4,AIUABA,1
...,...,...
179,URUBURETAMA,5
180,URUOCA,2
181,VARJOTA,3
182,VARZEA ALEGRE,2


In [7]:
nomes_municipios

,codigo_ibge,nome,latitude,longitude,UF,COD_MUNIC,NOTA_IDEB
0,2300101.0,ABAIARA,-7.34588,-39.0416,CE,2300101.0,6.1
1,2300150.0,ACARAPE,-4.22083,-38.7055,CE,2300150.0,5.4
2,2300200.0,ACARAU,-2.88769,-40.1183,CE,2300200.0,6.5
3,2300309.0,ACOPIARA,-6.08911,-39.4480,CE,2300309.0,6.0
4,2300408.0,AIUABA,-6.57122,-40.1178,CE,2300408.0,6.8
...,...,...,...,...,...,...,...
179,2313807.0,URUBURETAMA,-3.62316,-39.5107,CE,2313807.0,5.8
180,2313906.0,URUOCA,-3.30819,-40.5628,CE,2313906.0,8.3
181,2313955.0,VARJOTA,-4.19387,-40.4741,CE,2313955.0,6.8
182,2314003.0,VARZEA ALEGRE,-6.78264,-39.2942,CE,2314003.0,6.1


In [8]:
populacao

,NOME,POPULACAO
0,ABAIARA,11.737
1,ACARAPE,14.929
2,ACARAU,62.641
3,ACOPIARA,54.270
4,AIUABA,17.399
...,...,...
179,URUBURETAMA,21.850
180,URUOCA,13.840
181,VARJOTA,18.420
182,VARZEA ALEGRE,40.721


In [5]:
grafo_municipios

{'JIJOCA DE JERICOACOARA': ['CRUZ', 'CAMOCIM', 'BELA CRUZ'],
 'ACARAU': ['CRUZ', 'ITAREMA', 'BELA CRUZ', 'AMONTADA', 'MARCO', 'MORRINHOS'],
 'CRUZ': ['JIJOCA DE JERICOACOARA', 'ACARAU', 'BELA CRUZ'],
 'CAMOCIM': ['JIJOCA DE JERICOACOARA',
  'BARROQUINHA',
  'BELA CRUZ',
  'GRANJA',
  'MARCO'],
 'ITAREMA': ['ACARAU', 'AMONTADA'],
 'BARROQUINHA': ['CAMOCIM', 'CHAVAL', 'GRANJA'],
 'BELA CRUZ': ['JIJOCA DE JERICOACOARA', 'ACARAU', 'CRUZ', 'CAMOCIM', 'MARCO'],
 'CHAVAL': ['BARROQUINHA', 'GRANJA'],
 'AMONTADA': ['ACARAU',
  'ITAREMA',
  'ITAPIPOCA',
  'MORRINHOS',
  'SANTANA DO ACARAU',
  'MIRAIMA'],
 'GRANJA': ['CAMOCIM',
  'CHAVAL',
  'SENADOR SÁ',
  'MARTINOPOLE',
  'MARCO',
  'URUOCA',
  'VICOSA DO CEARA',
  'MORAUJO',
  'TIANGUA',
  'BARROQUINHA'],
 'MARTINOPOLE': ['GRANJA', 'SENADOR SA', 'URUOCA'],
 'MARCO': ['ACARAU',
  'CAMOCIM',
  'BELA CRUZ',
  'GRANJA',
  'SENADOR SA',
  'MORRINHOS'],
 'ITAPIPOCA': ['AMONTADA',
  'TRAIRI',
  'MIRAIMA',
  'IRAUCUBA',
  'ITAPAJE',
  'URUBURETAMA',
 

# 3. Processamentos

## 3.1 Ajuste no tipo da população

In [94]:
# Os dados de população viram como string, precisamos tirar os espaços, pontos e transformar em inteiro
populacao['POPULACAO'] = populacao['POPULACAO'].str.replace(' ','')
populacao['POPULACAO'] = populacao['POPULACAO'].str.replace('.','')
populacao['POPULACAO'] = populacao['POPULACAO'].astype('int')

<ipython-input-94-6d0967868838>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  populacao['POPULACAO'] = populacao['POPULACAO'].str.replace('.','')


## 3.2 Ajuste na ordem dos nomes dos municípios

In [6]:
# Ordenando o grafo para facilitar o laço que juntará todos os dados
grafo_mun_att = {}
for i in sorted(grafo_municipios.keys()):
    grafo_mun_att[i] = grafo_municipios[i]

In [12]:
grafo_mun_att

{'ABAIARA': ['BARRO', 'MISSAO VELHA', 'BREJO SANTO'],
 'ACARAPE': ['GUAIUBA', 'PACAJUS', 'REDENCAO', 'CHOROZINHO', 'BARREIRA'],
 'ACARAU': ['CRUZ', 'ITAREMA', 'BELA CRUZ', 'AMONTADA', 'MARCO', 'MORRINHOS'],
 'ACOPIARA': ['SOLONOPOLE',
  'PIQUET CARNEIRO',
  'DEPUTADO IRAPUAN PINHEIRO',
  'QUIXELO',
  'ARNEIROZ',
  'CATARINA',
  'IGUATU',
  'SABOEIRO',
  'JUCAS'],
 'AIUABA': ['PARAMBU',
  'ARNEIROZ',
  'CATARINA',
  'SABOEIRO',
  'ANTONINA DO NORTE',
  'CAMPOS SALES'],
 'ALCANTARAS': ['MERUOCA', 'MASSAPE', 'MORAUJO', 'COREARU', 'SOBRAL'],
 'ALTANEIRA': ['ASSARE', 'NOVA OLINDA', 'FARIAS BRITO'],
 'ALTO SANTO': ['MORADA NOVA',
  'TABULEIRO DO NORTE',
  'SAO JOAO DO JAGUARIBE',
  'JAGUARIBARA',
  'IRACEMA',
  'POTIRETAMA'],
 'AMONTADA': ['ACARAU',
  'ITAREMA',
  'ITAPIPOCA',
  'MORRINHOS',
  'SANTANA DO ACARAU',
  'MIRAIMA'],
 'ANTONINA DO NORTE': ['SABOEIRO',
  'AIUABA',
  'TARRAFAS',
  'CAMPOS SALES',
  'ASSARE'],
 'APUIARES': ['ITAPAJE',
  'PENTECOSTE',
  'TEJUCUOCA',
  'GENERAL SAMPAIO

## 3.3 Calculando CVLI por 100.000 habitantes

In [95]:
# Precisamos calcular a taxa de CVLI para evitar o enviesamento pelo número de habitantes
cvli_pop = round(populacao['POPULACAO']*cvli['QTD_CVLI']/100000, 2)

In [14]:
# Visualizando o resultado
cvli_pop

0      0.00
1      0.90
2      5.01
3      7.60
4      0.17
       ... 
179    1.09
180    0.28
181    0.55
182    0.81
183    1.83
Length: 184, dtype: float64

In [96]:
# Anexando o resultado no dataframe
cvli.assign(cvli_taxa=cvli_pop)

,MUNICIPIO,QTD_CVLI,cvli_taxa
0,ABAIARA,0,0.00
1,ACARAPE,6,0.90
2,ACARAU,8,5.01
3,ACOPIARA,14,7.60
4,AIUABA,1,0.17
...,...,...,...
179,URUBURETAMA,5,1.09
180,URUOCA,2,0.28
181,VARJOTA,3,0.55
182,VARZEA ALEGRE,2,0.81


In [97]:
cvli = cvli.assign(cvli_taxa=cvli_pop)

## 3.4 Deletando o id de GeoJSON

In [15]:
geojson_municipios.keys()

dict_keys(['type', 'features'])

In [16]:
geojson_municipios['features'][183]['properties']

{'id': '2314102', 'name': 'VICOSA DO CEARA', 'description': 'Viçosa do Ceará'}

In [17]:
geojson_municipios['features'][183].keys()

dict_keys(['type', 'properties', 'geometry'])

In [18]:
geojson_municipios['features'][183]['type']

'Feature'

In [8]:
for i in range(len(geojson_municipios['features'])):
    del geojson_municipios['features'][i]['properties']['id']

In [10]:
geojson_municipios['features'][183]['properties']

{'name': 'VICOSA DO CEARA', 'description': 'Viçosa do Ceará'}

## 3.5 Calculando o ângulo para o Vector Plot

In [62]:
# Função que recebe ar coordenadas de dois municipios
def calcular_angulo(x1,y1,x2,y2):
    x1 = abs(x1) # Transforma todos os valores em positivos
    x2 = abs(x2)
    y1 = abs(y1)
    y2 = abs(y2)
    
    x = abs(x2 - x1) # Calcula a distância relativa entre os dois pontos no eixo x
    y = abs(y2 - y1) # Calcula a distância relativa entre os dois pontos no eixo y
    
    # Dessa maneira teremos os lados de um triângulo e poderemos calcular o ângulo entre os pontos
    angulo = round(math.degrees(math.atan2(y,x))) # Calcula o arco tangente para encontrar o ângulo
    
    # Existem quatro casos para as posições dos maiores vizinhos onde cada caso tem um compensação diferente,
    # pois função math.atan2 realiza o cálculo seguindo a divisão normal dos quadrantes, e o parâmetro
    # de definição de ângulos do vega-lite funciona como um relógio.
    if x1 > x2 and y1 > y2: # Caso 1
        angulo = 90 - angulo
    elif x1 > x2 and y1 < y2: # Caso 2 
        angulo = angulo + 90
    elif x1 < x2 and y1 > y2: # Caso 3
        angulo += 270
    else: # x1 > x2 and y1 > y2              
        angulo = 90 - angulo + 180
    return angulo

In [64]:
# Calculando o ângulo do vizinho com maior nota
angulo_maior_nota = []
for i in grafo_mun_att.keys():
    notas = []
    for j in grafo_mun_att[i]:
        notas.append(float(nomes_municipios[nomes_municipios['nome']==j]['NOTA_IDEB']))
    index_maior_nota = notas.index(max(notas))
    vizinho_maior_nota = grafo_mun_att[i][index_maior_nota]
    
    x1 = float(nomes_municipios[nomes_municipios['nome']==i]['longitude'])
    y1 = float(nomes_municipios[nomes_municipios['nome']==i]['latitude'])
    x2 = float(nomes_municipios[nomes_municipios['nome']==vizinho_maior_nota]['longitude'])
    y2 = float(nomes_municipios[nomes_municipios['nome']==vizinho_maior_nota]['latitude'])
    
    angulo_maior_nota.append(calcular_angulo(x1,y1,x2,y2))

In [101]:
# Calculando o ângulo do vizinho com maior taxa de CVLI
angulo_maior_cvli_taxa = []
for i in grafo_mun_att.keys():
    cvli_taxa_list = []
    for j in grafo_mun_att[i]:
        cvli_taxa_list.append(int(cvli[cvli['MUNICIPIO']==j]['cvli_taxa']))
    index_maior_taxa_cvli = cvli_taxa_list.index(max(cvli_taxa_list))
    vizinho_maior_taxa_cvli = grafo_mun_att[i][index_maior_taxa_cvli]
    
    x1 = float(nomes_municipios[nomes_municipios['nome']==i]['longitude'])
    y1 = float(nomes_municipios[nomes_municipios['nome']==i]['latitude'])
    x2 = float(nomes_municipios[nomes_municipios['nome']==vizinho_maior_taxa_cvli]['longitude'])
    y2 = float(nomes_municipios[nomes_municipios['nome']==vizinho_maior_taxa_cvli]['latitude'])
    
    angulo_maior_cvli_taxa.append(calcular_angulo(x1,y1,x2,y2))

In [100]:
# Calculando o ângulo do vizinho com maior CVLI para teste
angulo_maior_cvli = []
for i in grafo_mun_att.keys():
    cvli_list = []
    for j in grafo_mun_att[i]:
        cvli_list.append(int(cvli[cvli['MUNICIPIO']==j]['QTD_CVLI']))
    index_maior_cvli = cvli_list.index(max(cvli_list))
    vizinho_maior_cvli = grafo_mun_att[i][index_maior_cvli]
    
    x1 = float(nomes_municipios[nomes_municipios['nome']==i]['longitude'])
    y1 = float(nomes_municipios[nomes_municipios['nome']==i]['latitude'])
    x2 = float(nomes_municipios[nomes_municipios['nome']==vizinho_maior_cvli]['longitude'])
    y2 = float(nomes_municipios[nomes_municipios['nome']==vizinho_maior_cvli]['latitude'])
    
    angulo_maior_cvli.append(calcular_angulo(x1,y1,x2,y2))

# 4. Juntando tudo

In [102]:
for i in range(len(geojson_municipios['features'])):
    nome = geojson_municipios['features'][i]['properties']['name']
    geojson_municipios['features'][i]['properties']['cvli'] = int(cvli['QTD_CVLI'][i])
    geojson_municipios['features'][i]['properties']['cvli_pop'] = cvli_pop[i]
    geojson_municipios['features'][i]['properties']['latitude'] = float(nomes_municipios['latitude'][i])
    geojson_municipios['features'][i]['properties']['longitude'] = float(nomes_municipios['longitude'][i])
    geojson_municipios['features'][i]['properties']['ideb'] = float(nomes_municipios['NOTA_IDEB'][i])
    geojson_municipios['features'][i]['properties']['angulo_nota'] = angulo_maior_nota[i]
    geojson_municipios['features'][i]['properties']['angulo_cvli'] = angulo_maior_cvli[i]
    geojson_municipios['features'][i]['properties']['angulo_taxa_cvli'] = angulo_maior_cvli_taxa[i]
    #geojson_municipios['features'][i]['properties']['vizinhos'] = grafo_mun_att[nome]
    

In [103]:
# Salvando os dados
with open('arquivos/data_final.geojson', 'w') as outfile: # Salvando os dados em formato geoJSON em um arquivo
    json.dump(geojson_municipios, outfile)

# Testes

In [18]:
grafo_mun_att['BREJO SANTO']

['BARRO',
 'MISSAO VELHA',
 'MAURITI',
 'ABAIARA',
 'PORTEIRAS',
 'JATI',
 'MILAGRES']

In [22]:
a = []
for j in grafo_mun_att['BREJO SANTO']:
        a.append(float(nomes_municipios[nomes_municipios['nome']==j]['NOTA_IDEB']))
index_maior_nota = a.index(max(a))
b = grafo_mun_att['BREJO SANTO'][index_maior_nota]

In [23]:
b

'ABAIARA'

In [53]:
float(nomes_municipios[nomes_municipios['nome']=='COREAU']['NOTA_IDEB'])

6.6

In [39]:
y1 = float(nomes_municipios[nomes_municipios['nome']=="ABAIARA"]['longitude'])
x1 = float(nomes_municipios[nomes_municipios['nome']=="ABAIARA"]['latitude'])
y2 = float(nomes_municipios[nomes_municipios['nome']=='BREJO SANTO']['longitude'])
x2 = float(nomes_municipios[nomes_municipios['nome']=='BREJO SANTO']['latitude'])

In [54]:
float(nomes_municipios[nomes_municipios['nome']=='BREJO SANTO']['longitude'])

-38.9799

In [35]:
abs(round(math.degrees(math.atan2(y2,x1))))+270

371

In [27]:
round(math.degrees(math.atan2(abs(y2),abs(x1))))

79

In [30]:
79 + 270

349

In [49]:
x1 = abs(x1)
x2 = abs(x2)
y1 = abs(y1)
y2 = abs(y2)
x = x2 - x1
y = y1 - y2

In [51]:
90 - round(math.degrees(math.atan2(y,x)))

66

In [45]:
11 + 90

101

In [17]:
66 + 90

156

In [14]:
round(math.degrees(math.atan2(5,8.66)))

30

In [58]:
x1 = float(nomes_municipios[nomes_municipios['nome']=="ABAIARA"]['longitude'])
y1 = float(nomes_municipios[nomes_municipios['nome']=="ABAIARA"]['latitude'])
x2 = float(nomes_municipios[nomes_municipios['nome']=='BREJO SANTO']['longitude'])
y2 = float(nomes_municipios[nomes_municipios['nome']=='BREJO SANTO']['latitude'])

In [63]:
calcular_angulo(x1,y1,x2,y2)

156

In [60]:
x = abs(abs(x2) - abs(x1))
y = abs(abs(y2) - abs(y1))
round(math.degrees(math.atan2(y,x)))

66

In [61]:
90+66

156

In [27]:
90-24

66